In [ ]:
import pandas as pd
import string
import os
import numpy as np
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from PIL import Image
import urllib
import requests
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import roc_curve, auc, classification_report
from sklearn.preprocessing import label_binarize
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score
import time
import glob
import seaborn as sns
import matplotlib.pyplot as plt
import operator
import folium
from itertools import cycle, islice
from pandas import options
import warnings
import pickle
import re
import nltk
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch
from matplotlib.pyplot import figure
from nltk.corpus import stopwords 
import nltk


nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

###Here is a Notebook that do the preprocess of our data just like in the first project. I save the splitted data in my drive so as to load them from the other Notebooks to save time.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## *Step 1:* Load and Read Data

In [ ]:
path = "/content/drive/My Drive/"

#Read data from csv file 
# df = pd.read_csv(path + 'SentimentTweets.csv') 
df = pd.read_csv(path + 'SentimentTweets.csv',nrows=500000) 
df = df.dropna()
df

,Unnamed: 0,target,id,date,flag,user,text
0,680949,0,2249621587,Fri Jun 19 22:41:08 PDT 2009,NO_QUERY,sukumarpant,#brokenpromises...
1,406741,0,2059003515,Sat Jun 06 16:03:21 PDT 2009,NO_QUERY,MTMSparrow,David Carradine so sad. Thai's law not sure i...
2,1337108,4,2017466467,Wed Jun 03 08:26:14 PDT 2009,NO_QUERY,itsmemcee,A @ 415 B @ 425. Tell your bro i say congrats!
3,1560887,4,2186457254,Mon Jun 15 18:52:04 PDT 2009,NO_QUERY,jdfreivald,@littlefluffycat Indeed.
4,1466295,4,2064458395,Sun Jun 07 06:19:20 PDT 2009,NO_QUERY,CrazyHan,Completed Race 4 Life in 58mins with girlies f...
...,...,...,...,...,...,...,...
499995,644302,0,2235992375,Fri Jun 19 02:35:10 PDT 2009,NO_QUERY,mojitoCyprus,@SkyNews they've finally managed to kill F1!
499996,528609,0,2195149293,Tue Jun 16 10:44:42 PDT 2009,NO_QUERY,rorossetto,@elvensapphire I'm brazilian so brazilian are ...
499997,1159941,4,1979388849,Sun May 31 02:50:09 PDT 2009,NO_QUERY,Princes_Doll,@ScherziNicole Aww thats so sweet! I joined a ...
499998,495969,0,2185406246,Mon Jun 15 17:20:03 PDT 2009,NO_QUERY,teagi_weegi,Woke up kinda depressed and sad


#### Take the columns that we want for Sentiment Analysis (text and target columns)

In [ ]:
df.drop(df.columns[0], axis=1, inplace=True)
df.drop(df.columns[1], axis=1, inplace=True)
df.drop(df.columns[1], axis=1, inplace=True)
df.drop(df.columns[1], axis=1, inplace=True)
df.drop(df.columns[1], axis=1, inplace=True)
df

,target,text
0,0,#brokenpromises...
1,0,David Carradine so sad. Thai's law not sure i...
2,4,A @ 415 B @ 425. Tell your bro i say congrats!
3,4,@littlefluffycat Indeed.
4,4,Completed Race 4 Life in 58mins with girlies f...
...,...,...
499995,0,@SkyNews they've finally managed to kill F1!
499996,0,@elvensapphire I'm brazilian so brazilian are ...
499997,4,@ScherziNicole Aww thats so sweet! I joined a ...
499998,0,Woke up kinda depressed and sad


## *Step 2:* Data Cleaning

### a) Lowercase of text set and get rid of the punctiations and symbols.


In [ ]:
df['text'] = df['text'].str.lower()

#Define function for removing special characters
def remove_special_characters(text, remove_digits=True):
    text=re.sub(r'[^a-zA-z0-9\s]','',text)
    return text

df['text'] = df['text'].apply(lambda x:''.join([i for i in x if i not in string.punctuation]))

#Apply function on review column
df['text']=df['text'].apply(remove_special_characters)


df

,target,text
0,0,brokenpromises
1,0,david carradine so sad thais law not sure if ...
2,4,a 415 b 425 tell your bro i say congrats
3,4,littlefluffycat indeed
4,4,completed race 4 life in 58mins with girlies f...
...,...,...
499995,0,skynews theyve finally managed to kill f1
499996,0,elvensapphire im brazilian so brazilian are an...
499997,4,scherzinicole aww thats so sweet i joined a wh...
499998,0,woke up kinda depressed and sad


Observe how many positive and negative comments we have

In [ ]:
df['target'].value_counts()

4    250800
0    249200
Name: target, dtype: int64

Replace the negative label of 4 to 1


In [ ]:
df['target'] = df['target'].replace(4,1)
display(df['target'])

0         0
1         0
2         1
3         1
4         1
         ..
499995    0
499996    0
499997    1
499998    0
499999    0
Name: target, Length: 500000, dtype: int64

### 2) Remove tags and links 

In [ ]:
import re
#Remove tags and links 
tag = re.compile(r'<[^>]+>')

df['text'] = df['text'].apply(lambda x: tag.sub('', x)) #removing html labels

df['text'] = df['text'].replace(r'http\S+', '', regex=True).replace(r'www.\S+', '', regex=True).replace(r'http\S+', '', regex=True).replace(r'"', '', regex=True)

df

,target,text
0,0,brokenpromises
1,0,david carradine so sad thais law not sure if ...
2,1,a 415 b 425 tell your bro i say congrats
3,1,littlefluffycat indeed
4,1,completed race 4 life in 58mins with girlies f...
...,...,...
499995,0,skynews theyve finally managed to kill f1
499996,0,elvensapphire im brazilian so brazilian are an...
499997,1,scherzinicole aww thats so sweet i joined a wh...
499998,0,woke up kinda depressed and sad


### 3) Tokenization and Lemmanization


In [ ]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    return " ".join([lemmatizer.lemmatize(w, pos="v") for w in w_tokenizer.tokenize(text)])

df['text'] = df.text.apply(lemmatize_text).copy()

df

,target,text
0,0,brokenpromises
1,0,david carradine so sad thais law not sure if i...
2,1,a 415 b 425 tell your bro i say congrats
3,1,littlefluffycat indeed
4,1,complete race 4 life in 58mins with girlies fr...
...,...,...
499995,0,skynews theyve finally manage to kill f1
499996,0,elvensapphire im brazilian so brazilian be any...
499997,1,scherzinicole aww thats so sweet i join a whil...
499998,0,wake up kinda depress and sad


## *Step 3*: Split dataset into train, test and validation set

I choose to split the dataset to 99%-1% because is too big

In [ ]:
x = pd.DataFrame(df, columns = ['text']) 
y = pd.DataFrame(df, columns = ['target']) 

# Split dataset to train and test set.
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.005, random_state=42)

# Split train dataset to train and validation set.
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.005, random_state=42)

print("Shape of x_train: ", X_train.shape)
print("Shape of y_train: ", Y_train.shape)
print("Shape of x_test:  ", X_test.shape)
print("Shape of y_test:  ", Y_test.shape)
print("Shape of x_val:  ", X_val.shape)
print("Shape of y_val:  ", Y_val.shape)

Shape of x_train:  (495012, 1)
Shape of y_train:  (495012, 1)
Shape of x_test:   (2500, 1)
Shape of y_test:   (2500, 1)
Shape of x_val:   (2488, 1)
Shape of y_val:   (2488, 1)


## *Step 4*: Save the data in order to use them in other Notebooks

In [ ]:
x_train_file = open('/content/drive/My Drive/X_train.pkl', 'wb')
pickle.dump(X_train,x_train_file)
x_train_file.close()

x_test_file = open('/content/drive/My Drive/X_test.pkl', 'wb')
pickle.dump(X_test,x_test_file)
x_test_file.close()

y_train_file = open('/content/drive/My Drive/Y_train.pkl', 'wb')
pickle.dump(Y_train,y_train_file)
y_train_file.close()

y_test_file = open('/content/drive/My Drive/Y_test.pkl', 'wb')
pickle.dump(Y_test,y_test_file)
y_test_file.close()

x_val_file = open('/content/drive/My Drive/X_val.pkl', 'wb')
pickle.dump(X_val,x_val_file)
x_val_file.close()

y_val_file = open('/content/drive/My Drive/Y_val.pkl', 'wb')
pickle.dump(Y_val,y_val_file)
y_val_file.close()